In [1]:
import gc

import numpy as np
import pandas as pd
import spacy
import textacy
import utils_clean
import utils_text
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from tqdm import tqdm

/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

train.fillna('CVxTz', inplace = True)
train = utils_clean.apply_wordlist_clean(train)
train = utils_clean.apply_basic_cleaning(train)

test.fillna('CVxTz', inplace = True)
test = utils_clean.apply_wordlist_clean(test)
test = utils_clean.apply_basic_cleaning(test)

100%|██████████| 2/2 [00:07<00:00,  3.88s/it]


In [3]:
train.comment_text = train.comment_text.apply(lambda x: textacy.preprocess.preprocess_text(x, fix_unicode = True,
                                                                            lowercase = True,
                                                                            no_contractions = True,
                                                                            transliterate = True))

test.comment_text = test.comment_text.apply(lambda x: textacy.preprocess.preprocess_text(x, fix_unicode = True,
                                                                            lowercase = True,
                                                                            no_contractions = True,
                                                                            transliterate = True))

In [4]:
nlp = spacy.load('en')
stops = stopwords.words("english")


q1 = []
for doc in tqdm(nlp.pipe(train['comment_text'], n_threads=8, batch_size=128)):
    word_list = ([c.lemma_.strip() for c in doc if c.lemma_ not in stops])
    q1.append(' '.join(i for i in word_list))


q2 = []
for doc in tqdm(nlp.pipe(test['comment_text'], n_threads=8, batch_size=128)):
    word_list = ([c.lemma_.strip() for c in doc if c.lemma_ not in stops])
    q2.append(' '.join(i for i in word_list))

159571it [28:00, 94.94it/s]
153164it [24:34, 103.84it/s]


In [5]:
q1 = pd.Series(q1)
q2 = pd.Series(q2)

train.comment_text = q1
test.comment_text = q2

In [6]:
train.to_pickle('../data/train_textacy_clean.pkl')
test.to_pickle('../data/test_textacy_clean.pkl')

In [7]:
train.comment_text = train.comment_text.apply(lambda x: (' '.join([utils_clean.correction(i) for i in utils_text.word_tokenize(x)])))
test.comment_text = test.comment_text.apply(lambda x: (' '.join([utils_clean.correction(i) for i in utils_text.word_tokenize(x)])))

In [8]:
train.to_pickle('../data/train_textacy_fullclean.pkl')
test.to_pickle('../data/test_textacy_fullclean.pkl')